In [ ]:
#Use PointbestDiffernce(Plant) to find best G and alpha using point differnce
    #input: networkx arbor
    #output:
        #closest optimal networkx arbor
        #Closest G
        #closest alpha
        
#Use ClosestEuclid(originalPlant) to find best G and alpha using point diffence
    #input: netowrkx arbor
    #output:
        #front: networkx object of all nodes with coordinates (conduction delay, wiring cost) for diffent Gs and alphas
        #orignal plant
        #the best euclid distance
        #bestNodeArray
            #the cordinate which is (conducton delay, wiringcost)
            #the best G value
            #best alpha value
        #x: conduction delay
        #y:wiring cost

In [5]:
import numpy as np
%run NetworkxConverstion.ipynb
%run WringCostAndConductionDelay.ipynb

In [2]:
#point by point best differnce

In [3]:
#find best G and alpha using point differnce
    #input: networx arbor
    #output:
        #closest optimal networkx arbor
        #Closest G
        #closest alpha
def PointbestDiffernce(Plant):
    Grange = np.arange(-2,2.1,.1)
    bestG = -2
    aRange = np.arange(0,1.1,.1)
    bestA =0
    Plant2 = optimalArbor(Plant, bestG, bestA)
    bestDif = pointDif(Plant,Plant2)
    for G in Grange:
        for alpha in aRange:
            Plant2 = optimalArbor(Plant, G, alpha)
            dif = findDiffernce(Plant,Plant2,bestDif)
            if dif < bestDif:
                bestDif = dif
                bestA = alpha
                bestG =G
        print(G)#for me to see if it is still working
    bestPlant = optimalArbor(Plant,bestG, bestA)
    return(bestPlant,bestG, bestA)

In [4]:
#saves a little time in that it bales if the current dif is too large
#basicly the same as pointDif
def findDiffernce(PlantOg,PlantNew,bestDif):
    dif =0.0
    for node in PlantOg.nodes():
        if PlantOg.nodes[node]['label'] != 'lateral root':
            if PlantOg.nodes[node]['label'] != 'lateral root tip':
                if PlantOg.nodes[node]['label'] != 'lateral root base':
                    continue
        rootNumber = PlantOg.nodes[node]['root number']
        if rootNumber ==0:
            continue
        ox,oy = PlantOg.nodes[node]['coordinate']
        for edge in PlantNew.edges():
            if PlantNew.edges[edge]['label'] != 'lateral root':
                continue
            elif PlantNew.edges[edge]['root number'] != rootNumber:
                continue
            nxs = PlantNew.edges[edge]['xs']
            nys = PlantNew.edges[edge]['ys']
            bestDx = np.absolute(nxs[0]-ox)
            besti = 0
            for i in range(1, len(nxs)):
                dx = np.absolute(nxs[i] -ox)
                if dx < bestDx:
                    bestDx = dx
                    besti = i
            dy = np.absolute(nys[besti] -oy)
            dif = dif + dy 
            if dif > bestDif:
                break
        if dif> bestDif:
            break
    return(dif)

In [6]:
#Euclid Distance

In [7]:
#finds best G and alpha using point diffence
    #input: netowrkx arbor
    #output:
        #front: networkx object of all nodes with coordinates (conduction delay, wiring cost) for diffent Gs and alphas
        #orignal plant
        #the best euclid distance
        #bestNodeArray
            #the cordinate which is (conducton delay, wiringcost)
            #the best G value
            #best alpha value
        #x: conduction delay
        #y:wiring cost
def ClosestEuclid(originalPlant):
    Grange = np.arange(-2,2.1,.1)
    Arange = np.arange(0,1.1,.1)
    front = nx.Graph()
    cds=[]
    wcs=[]
    c =0
    y = calculateWC(originalPlant)
    x = calculateCD(originalPlant)
    bestDis = 10**20000
    for G in Grange:
        print(G) #just to see how long it takes
        for a in Arange:
            newplant = optimalArbor(originalPlant, G, a)
            cd =calculateWC(newplant)
            wc = calculateCD(newplant)
            cds.append(cd)
            wcs.append(wc)
            front.add_node(c)
            front._node[c]['coordinate']=(cd,wc)
            front._node[c]['G'] =G
            front._node[c]['alpha'] =a
            c+=1
            dis = np.sqrt((x-cd)**2 +(y-wc)**2)
            if dis < bestDis:
                bestDis = dis
                bestNode = c
    bestNodeArray = [front._node[bestNode]['coordinate'], front._node[bestNode]['G'],front._node[bestNode]['alpha']]
    return(front, originalPlant, bestDis,bestNodeArray, x,y)
    #front: networkx object of all nodes with coordinates (conduction delay, wiring cost) for diffent Gs and alphas
    #orignal plant
    #the best euclid distance
    #bestNodeArray
        #the cordinate which is (conducton delay, wiringcost)
        #the best G value
        #best alpha value
    #x: conduction delay
    #y:wiring cost
        